# 주가 데이터를 일반적인 지도학습으로 학습합니다. Regression 모델과 Classification 모델을 시도해 봅니다.

In [1]:
import keras.backend.tensorflow_backend as K
from keras.layers import Input, Dense, LSTM, GRU, Flatten, Lambda
from keras.models import Model
from keras.callbacks import ModelCheckpoint

from os.path import isfile, join
from os import listdir
import numpy as np

Using TensorFlow backend.


# 1. Data Loader & Step estimator & train/test splitter
## 링크: http://121.140.2.142:8888/notebooks/test_notes/preprocess/stockdata_preprocess_basic.ipynb

In [29]:
def data_loader(all_stock_data, batch_size, pre_data_length, pro_data_length, process_data):
    num_company = len(all_stock_data)
    num_features = all_stock_data[0].shape[1]
    company_idx = list(range(num_company))
    current_idx = np.array([0 for _ in range(num_company)])
    
    def get_batch(all_stock_data, current_idx, batch_company_idx, pre_data_length, pro_data_length):
        batch_company = [all_stock_data[idx] for idx in batch_company_idx]
        
        # get cur pos of each data in mini batch
        cur_pos = current_idx[batch_company_idx]
        x_batch = np.empty(shape=(0, pre_data_length, num_features))
        y_batch = np.empty(shape=(0, pro_data_length, num_features))
        
        for company, pos in zip(batch_company, cur_pos):
            _x_batch = company[pos: pos+pre_data_length].reshape(1, pre_data_length, num_features)
            _y_batch = company[pos+pre_data_length: pos+pre_data_length+pro_data_length].reshape(1, pro_data_length, num_features)
            
#             print(_new_x_batch)
            x_batch = np.concatenate([x_batch, _x_batch], axis=0)
            y_batch = np.concatenate([y_batch, _y_batch], axis=0)
        
        # update for next pos
        new_pos = [c + 1 for c in cur_pos]
        current_idx[batch_company_idx] = new_pos

        # check if next pos possible, and remove from company_idx if not possible
        for idx, (company, pos) in enumerate(zip(batch_company, new_pos)):
            if company.shape[0] < pos + pre_data_length + pro_data_length:
#                 print(company[0, 7], ' finished at pos:', pos)
                company_idx.remove(batch_company_idx[idx])
                np.delete(current_idx, batch_company_idx[idx])
        
        return x_batch, y_batch
    
    count = 0
    while True:
        while len(company_idx) > 0:
#             if count == 0:
#                 print('New Epoch begins', len(company_idx))
            count += 1
            # random pick from all batch
            if len(company_idx) >= batch_size:
                batch_company_idx = sample(company_idx, batch_size)


                x_batch, y_batch = get_batch(all_stock_data=all_stock_data, current_idx=current_idx, 
                                             pre_data_length=pre_data_length, pro_data_length=pro_data_length,
                                             batch_company_idx=batch_company_idx)

            else: # if number of company less than batch, then company should go for parallel
                batch_company_idx = company_idx[:]
                x_batch = list()
                y_batch = list()
                idx = 0
                while len(x_batch) < batch_size and len(company_idx) > 0:
                    if idx >= len(company_idx):
                        idx = 0
                    batch_company_idx = company_idx[idx]
                    idx += 1
                    idx = idx % len(company_idx)
                    _x_batch, _y_batch = get_batch(all_stock_data=all_stock_data, current_idx=current_idx, 
                                                   pre_data_length=pre_data_length, pro_data_length=pro_data_length,
                                                   batch_company_idx=[batch_company_idx])
                    x_batch.append(_x_batch)
                    y_batch.append(_y_batch)

                x_batch = np.concatenate(x_batch, axis=0)
                y_batch = np.concatenate(y_batch, axis=0)

            # 마지막 처리를 해줍니다.
            x_batch, y_batch = process_data(x_batch, y_batch)

            yield x_batch, y_batch
        
        company_idx = list(range(num_company))
        current_idx = np.array([0 for _ in range(num_company)])

In [67]:
# calculating total estimated steps
def get_estimated_steps(all_stock_data, pre_data_length, pro_data_length, batch_size):
    
    all_length = pre_data_length + pro_data_length

    total_steps = 0
    for stock_data in all_stock_data:
        total_steps += (stock_data.shape[0] - all_length + 1)
    #     print(total_steps)

    return total_steps, int(np.ceil(total_steps / batch_size))

In [4]:
def split_train_test_by_year(all_stock_data, year): # year 이전은 train, year 해당 년 및 이후는 test
    train_stock_data = list()
    test_stock_data = list()
    for stock_data in all_stock_data:
        indices = np.where(stock_data[:, 0] >= (year * 10000))[0]
        test_stock_data.append(stock_data[indices][:, 1:])
        
        indices = np.where(stock_data[:, 0] < (year * 10000))[0]
        train_stock_data.append(stock_data[indices][:, 1:])
    
    test_stock_data = [stock_data for stock_data in test_stock_data if stock_data.shape[0] > 0]
    train_stock_data = [stock_data for stock_data in train_stock_data if stock_data.shape[0] > 0]
    
    return train_stock_data, test_stock_data

In [5]:
"""
1. '날짜': 'date', 2. '종가': 'final_price', 3. '전일비': 'compare_to_prior', 4. '시가': 'start_price',
5. '고가': 'highest_price', 6. '저가': 'lowest_price', 7. '거래량': 'num_of_traded', 8. '회사이름': 'company_name'
9. df_temp = Features.fnMACD(df_temp)
    df_temp = Features.fnBolingerBand(df_temp)
    df_temp = Features.fnRSI(df_temp)
    df_temp = Features.fnStoch(df_temp)
    df_temp = Features.change_prior_to(df_temp)
    df_temp = Features.fnMA(df_temp, m_N=[5, 20, 60, 120, 240])
"""

def load_data(root_path):
    all_stock_data = list()
    for filename in listdir(root_path):
        if not isfile(join(root_path, filename)) or '.npy' not in filename:
            continue
        print(filename)
        stock_data = np.load(join(root_path, filename))
        all_str_dates = stock_data[:, 0]
        all_str_dates = [int(s.replace('.', '')) for s in all_str_dates]
        stock_data[:, 0] = all_str_dates
        stock_data = stock_data[::-1]
        stock_data = stock_data[:, :-1] # remove and company name index
        all_stock_data.append(stock_data)
    
    return all_stock_data

# 2. Load Data and split train & test set

In [6]:
root_path = '/data1/stock_data_temp'

In [7]:
all_stock_data = load_data(root_path=root_path)
train_stock_data, test_stock_data = split_train_test_by_year(all_stock_data=all_stock_data, year=2017) # this will also remove date

경방.npy
경보제약.npy
경동도시가스.npy
경동인베스트.npy


In [8]:
train_stock_data[0].shape

(5172, 6)

In [9]:
for stock_data in train_stock_data:
    print(stock_data.shape)

(5172, 6)
(374, 6)
(4951, 6)


# 3. Model train and test class

## 3-1. t+1일의 가격으로 regression용 데이터와 classification용 데이터를 만들어주는 로직을 만들어 data_loader의 process_data로 넣을 수 있도록 합니다.

In [105]:
# regression example
def process_regression_data(x_batch, y_batch):
    
    y_batch = y_batch[:, :, 0] # final price index = 1
    
    return x_batch, y_batch

def process_classification_data(x_batch, y_batch):
    
    y_batch = y_batch[:, :, 2] # final price index = 1
    y_batch[np.where(y_batch <= 0)[0], 0] = 0
    y_batch[np.where(y_batch > 0)[0], 0] = 1
    
    return x_batch, y_batch

## 3-2. 주어진 모델에 대해 학습하고 테스트 할 수 있는 클래스를 만들어 줍니다.

In [106]:
class StockPredictionActivator:
    def __init__(self, model):
        self.model = model
        print(self.model.summary())
        
    def train(self, train_data, validation_data=None, epochs=50, batch_size=64, verbose=1, validation_split=0.1, **kwargs):
#         print(len(args))
        
        pre_data_length = kwargs['pre_data_length']
        pro_data_length = kwargs['pro_data_length']
        process_data = kwargs['process_data']
        step_estimator = kwargs['step_estimator']
        generator = kwargs['generator']
        train_steps, train_steps_per_epoch = step_estimator(train_data, pre_data_length, pro_data_length, batch_size)
        val_steps, val_steps_per_epoch = step_estimator(validation_data, pre_data_length, pro_data_length, batch_size)
        
        print('Trains steps:', train_steps_per_epoch, 'Test steps:', val_steps_per_epoch)
        
#         mc = ModelCheckpoint('./save/s2s_{epoch:03d}.h5', save_weights_only=True, period=5)
        
        self.model.fit_generator(generator=generator(all_stock_data=train_data, 
                                                       batch_size=batch_size, 
                                                       pre_data_length=pre_data_length, 
                                                       pro_data_length=pro_data_length,
                                                       process_data=process_data),
                                 steps_per_epoch=train_steps_per_epoch,
                                 validation_data=generator(all_stock_data=validation_data, 
                                                       batch_size=batch_size, 
                                                       pre_data_length=pre_data_length, 
                                                       pro_data_length=pro_data_length,
                                                       process_data=process_data),
                                 validation_steps=val_steps_per_epoch,
                                 epochs=epochs,
                                 verbose=verbose,
#                                  callbacks=[mc]
                                )
        print('')
    
    def test(self, test_data, tester, **kwargs):
        pre_data_length = kwargs['pre_data_length']
        pro_data_length = kwargs['pro_data_length']
        process_data = kwargs['process_data']
        step_estimator = kwargs['step_estimator']
        
        test_steps_per_epoch = step_estimator(test_data, pre_data_length, pro_data_length, batch_size)
        tester(test_data)
        

## 3-3. lstm regression 심플 모델을 생성하고 테스트합니다.

In [113]:
def get_simple_lstm(pre_data_length, num_features, output_dim):
    inputs = Input(shape=(pre_data_length, num_features))
    lstm_layer = LSTM(64)
    lstm_outputs = lstm_layer(inputs)
#     flat_lstm_outputs = Flatten()(lstm_outputs)
    outputs = Dense(output_dim)(lstm_outputs)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
    
    return model

In [114]:
pre_data_length = 20
pro_data_length = 1
process_data = process_regression_data

In [115]:
sp_activator = StockPredictionActivator(get_simple_lstm(pre_data_length=pre_data_length, num_features=6, output_dim=1))
sp_activator.train(train_data=train_stock_data, validation_data=test_stock_data, 
                   epochs=1000, batch_size=64, verbose=1, validation_split=0.1,
                  pre_data_length=pre_data_length, pro_data_length=pro_data_length, process_data=process_data,
                  step_estimator=get_estimated_steps, generator=data_loader)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 20, 6)             0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                18176     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 18,241
Trainable params: 18,241
Non-trainable params: 0
_________________________________________________________________
None
Trains steps: 164 Test steps: 22
Epoch 1/1000
164/164 [==============================] - 4s 21ms/step - loss: 7127609037.3189 - acc: 0.0000e+00 - val_loss: 1294052659.7516 - val_acc: 0.0000e+00
Epoch 2/1000
164/164 [==============================] - 2s 15ms/step - loss: 7127268613.4388 - acc: 0.0000e+00 - val_loss: 1293919356.8678 - val_acc: 0.0000e+00
Epoch 3/1000
164/164 [====================

Epoch 47/1000
164/164 [==============================] - 2s 15ms/step - loss: 7110509351.2261 - acc: 0.0000e+00 - val_loss: 1287071256.3097 - val_acc: 0.0000e+00
Epoch 48/1000
164/164 [==============================] - 2s 15ms/step - loss: 7110137187.6185 - acc: 0.0000e+00 - val_loss: 1286919611.5121 - val_acc: 0.0000e+00
Epoch 49/1000
164/164 [==============================] - 2s 15ms/step - loss: 7109765200.0158 - acc: 0.0000e+00 - val_loss: 1286767924.8269 - val_acc: 0.0000e+00
Epoch 50/1000
164/164 [==============================] - 2s 15ms/step - loss: 7109393130.9474 - acc: 0.0000e+00 - val_loss: 1286616286.0131 - val_acc: 0.0000e+00
Epoch 51/1000
164/164 [==============================] - 2s 15ms/step - loss: 7109021154.2719 - acc: 0.0000e+00 - val_loss: 1286464621.4405 - val_acc: 0.0000e+00
Epoch 52/1000
164/164 [==============================] - 2s 15ms/step - loss: 7108649011.3468 - acc: 0.0000e+00 - val_loss: 1286312997.5866 - val_acc: 0.0000e+00
Epoch 53/1000
164/164 [=====

164/164 [==============================] - 2s 15ms/step - loss: 7091927217.8263 - acc: 0.0000e+00 - val_loss: 1279501721.7122 - val_acc: 0.0000e+00
Epoch 98/1000
164/164 [==============================] - 2s 15ms/step - loss: 7091556141.4656 - acc: 0.0000e+00 - val_loss: 1279350672.0351 - val_acc: 0.0000e+00
Epoch 99/1000
164/164 [==============================] - 2s 15ms/step - loss: 7091184869.8770 - acc: 0.0000e+00 - val_loss: 1279199587.1088 - val_acc: 0.0000e+00
Epoch 100/1000
164/164 [==============================] - 2s 15ms/step - loss: 7090813841.4209 - acc: 0.0000e+00 - val_loss: 1279048509.3353 - val_acc: 0.0000e+00
Epoch 101/1000
164/164 [==============================] - 2s 15ms/step - loss: 7090442717.1544 - acc: 0.0000e+00 - val_loss: 1278897392.6662 - val_acc: 0.0000e+00
Epoch 102/1000
164/164 [==============================] - 2s 15ms/step - loss: 7090071549.3753 - acc: 0.0000e+00 - val_loss: 1278746450.6998 - val_acc: 0.0000e+00
Epoch 103/1000
164/164 [===============

164/164 [==============================] - 2s 15ms/step - loss: 7073392093.2526 - acc: 0.0000e+00 - val_loss: 1271961874.3258 - val_acc: 0.0000e+00
Epoch 148/1000
164/164 [==============================] - 2s 15ms/step - loss: 7073021881.3019 - acc: 0.0000e+00 - val_loss: 1271811391.6728 - val_acc: 0.0000e+00
Epoch 149/1000
164/164 [==============================] - 2s 15ms/step - loss: 7072651733.6454 - acc: 0.0000e+00 - val_loss: 1271660884.5698 - val_acc: 0.0000e+00
Epoch 150/1000
164/164 [==============================] - 2s 15ms/step - loss: 7072281467.7422 - acc: 0.0000e+00 - val_loss: 1271510463.5793 - val_acc: 0.0000e+00
Epoch 151/1000
164/164 [==============================] - 2s 15ms/step - loss: 7071911268.5707 - acc: 0.0000e+00 - val_loss: 1271360027.6289 - val_acc: 0.0000e+00
Epoch 152/1000
164/164 [==============================] - 2s 15ms/step - loss: 7071541087.3521 - acc: 0.0000e+00 - val_loss: 1271209530.6706 - val_acc: 0.0000e+00
Epoch 153/1000
164/164 [=============

164/164 [==============================] - 2s 15ms/step - loss: 7054903434.3694 - acc: 0.0000e+00 - val_loss: 1264451644.1665 - val_acc: 0.0000e+00
Epoch 198/1000
164/164 [==============================] - 2s 15ms/step - loss: 7054534165.3674 - acc: 0.0000e+00 - val_loss: 1264301672.9993 - val_acc: 0.0000e+00
Epoch 199/1000
164/164 [==============================] - 2s 15ms/step - loss: 7054164865.6320 - acc: 0.0000e+00 - val_loss: 1264151801.0811 - val_acc: 0.0000e+00
Epoch 200/1000
164/164 [==============================] - 2s 15ms/step - loss: 7053795603.2654 - acc: 0.0000e+00 - val_loss: 1264001936.9701 - val_acc: 0.0000e+00
Epoch 201/1000
164/164 [==============================] - 2s 15ms/step - loss: 7053426368.8982 - acc: 0.0000e+00 - val_loss: 1263852045.2768 - val_acc: 0.0000e+00
Epoch 202/1000
164/164 [==============================] - 2s 15ms/step - loss: 7053057163.9954 - acc: 0.0000e+00 - val_loss: 1263702212.9087 - val_acc: 0.0000e+00
Epoch 203/1000
164/164 [=============

...

164/164 [==============================] - 2s 15ms/step - loss: 6836677134.0177 - acc: 0.0000e+00 - val_loss: 1176631691.8744 - val_acc: 0.0000e+00
Epoch 798/1000
164/164 [==============================] - 2s 15ms/step - loss: 6836319018.5499 - acc: 0.0000e+00 - val_loss: 1176488839.8072 - val_acc: 0.0000e+00
Epoch 799/1000
164/164 [==============================] - 2s 15ms/step - loss: 6835961022.3062 - acc: 0.0000e+00 - val_loss: 1176346047.5793 - val_acc: 0.0000e+00
Epoch 800/1000
164/164 [==============================] - 2s 15ms/step - loss: 6835602934.5460 - acc: 0.0000e+00 - val_loss: 1176203201.4960 - val_acc: 0.0000e+00
Epoch 801/1000
164/164 [==============================] - 2s 15ms/step - loss: 6835244879.9578 - acc: 0.0000e+00 - val_loss: 1176060463.1234 - val_acc: 0.0000e+00
Epoch 802/1000
164/164 [==============================] - 2s 15ms/step - loss: 6834886828.3940 - acc: 0.0000e+00 - val_loss: 1175917654.1125 - val_acc: 0.0000e+00
Epoch 803/1000
164/164 [=============

164/164 [==============================] - 2s 15ms/step - loss: 6818795763.2770 - acc: 0.0000e+00 - val_loss: 1169505235.8685 - val_acc: 0.0000e+00
Epoch 848/1000
164/164 [==============================] - 2s 15ms/step - loss: 6818438654.6625 - acc: 0.0000e+00 - val_loss: 1169363059.7049 - val_acc: 0.0000e+00
Epoch 849/1000
164/164 [==============================] - 2s 15ms/step - loss: 6818081517.0714 - acc: 0.0000e+00 - val_loss: 1169220859.0913 - val_acc: 0.0000e+00
Epoch 850/1000
164/164 [==============================] - 2s 15ms/step - loss: 6817724511.6811 - acc: 0.0000e+00 - val_loss: 1169078691.9036 - val_acc: 0.0000e+00
Epoch 851/1000
164/164 [==============================] - 2s 15ms/step - loss: 6817367477.2144 - acc: 0.0000e+00 - val_loss: 1168936511.5793 - val_acc: 0.0000e+00
Epoch 852/1000
164/164 [==============================] - 2s 15ms/step - loss: 6817010456.8950 - acc: 0.0000e+00 - val_loss: 1168794363.5121 - val_acc: 0.0000e+00
Epoch 853/1000
164/164 [=============

164/164 [==============================] - 2s 15ms/step - loss: 6800963298.5921 - acc: 0.0000e+00 - val_loss: 1162409111.3747 - val_acc: 0.0000e+00
Epoch 898/1000
164/164 [==============================] - 2s 15ms/step - loss: 6800607093.9017 - acc: 0.0000e+00 - val_loss: 1162267559.0825 - val_acc: 0.0000e+00
Epoch 899/1000
164/164 [==============================] - 2s 15ms/step - loss: 6800250824.4286 - acc: 0.0000e+00 - val_loss: 1162125910.5332 - val_acc: 0.0000e+00
Epoch 900/1000
164/164 [==============================] - 2s 15ms/step - loss: 6799894638.2754 - acc: 0.0000e+00 - val_loss: 1161984306.2089 - val_acc: 0.0000e+00
Epoch 901/1000
164/164 [==============================] - 2s 15ms/step - loss: 6799538491.2450 - acc: 0.0000e+00 - val_loss: 1161842804.7801 - val_acc: 0.0000e+00
Epoch 902/1000
164/164 [==============================] - 2s 15ms/step - loss: 6799182435.6323 - acc: 0.0000e+00 - val_loss: 1161701179.5121 - val_acc: 0.0000e+00
Epoch 903/1000
164/164 [=============

164/164 [==============================] - 2s 15ms/step - loss: 6783176321.4310 - acc: 0.0000e+00 - val_loss: 1155342037.0840 - val_acc: 0.0000e+00
Epoch 948/1000
164/164 [==============================] - 2s 15ms/step - loss: 6782820975.8827 - acc: 0.0000e+00 - val_loss: 1155201014.7436 - val_acc: 0.0000e+00
Epoch 949/1000
164/164 [==============================] - 2s 15ms/step - loss: 6782465862.3417 - acc: 0.0000e+00 - val_loss: 1155059942.1943 - val_acc: 0.0000e+00
Epoch 950/1000
164/164 [==============================] - 2s 15ms/step - loss: 6782110540.6967 - acc: 0.0000e+00 - val_loss: 1154918933.6450 - val_acc: 0.0000e+00
Epoch 951/1000
164/164 [==============================] - 2s 15ms/step - loss: 6781755343.4455 - acc: 0.0000e+00 - val_loss: 1154777896.9993 - val_acc: 0.0000e+00
Epoch 952/1000
164/164 [==============================] - 2s 15ms/step - loss: 6781400085.8991 - acc: 0.0000e+00 - val_loss: 1154636941.6508 - val_acc: 0.0000e+00
Epoch 953/1000
164/164 [=============

164/164 [==============================] - 2s 15ms/step - loss: 6765434387.7780 - acc: 0.0000e+00 - val_loss: 1148303562.0044 - val_acc: 0.0000e+00
Epoch 998/1000
164/164 [==============================] - 2s 15ms/step - loss: 6765080006.3509 - acc: 0.0000e+00 - val_loss: 1148163114.4953 - val_acc: 0.0000e+00
Epoch 999/1000
164/164 [==============================] - 2s 15ms/step - loss: 6764725690.7792 - acc: 0.0000e+00 - val_loss: 1148022666.9861 - val_acc: 0.0000e+00
Epoch 1000/1000
164/164 [==============================] - 2s 15ms/step - loss: 6764371344.5729 - acc: 0.0000e+00 - val_loss: 1147882215.3163 - val_acc: 0.0000e+00



In [118]:
print('ahahaha')

ahahaha


In [116]:
def test(model, test_data, tester, batch_size=64, **kwargs):
    pre_data_length = kwargs['pre_data_length']
    pro_data_length = kwargs['pro_data_length']
    process_data = kwargs['process_data']
    step_estimator = kwargs['step_estimator']
    generator = kwargs['generator']
    test_steps, test_steps_per_epoch = step_estimator(test_data, pre_data_length, pro_data_length, batch_size)
    print('Total Samples:', test_steps)

    """
    모든 회사를 한꺼번에 넣으면 각 미니배치마다 랜덤 회사를 가지고 오기 때문에, predict 된 내역과 실제 y를 비교 불가능하게 된다.
    따라서 한 회사씩 predict하여 그 순서가 회사 -> 시간순으로 예측되도록 한다.
    """
    all_predicted = list()
    for td in test_data:
        test_steps, test_steps_per_epoch = step_estimator([td], pre_data_length, pro_data_length, batch_size)
        predicted = model.predict_generator(generator=generator(all_stock_data=[td], 
                                                                batch_size=batch_size, 
                                                                pre_data_length=pre_data_length, 
                                                                pro_data_length=pro_data_length,
                                                                process_data=process_data),
                                            steps=test_steps_per_epoch)
        all_predicted.extend(predicted)
    
    tester(test_data, predicted, pre_data_length=pre_data_length, pro_data_length=pro_data_length)
    return all_predicted

In [117]:
def regression_tester(test_data, predicted, **kwargs):
    pre_data_length = kwargs['pre_data_length']
    pro_data_length = kwargs['pro_data_length']
    
    y_data = list()
    for td in test_data:
        y_data.extend(td[pre_data_length:, 0])
    
    for y, y_hat in zip(y_data, predicted):
        print(y, y_hat)
    
all_predicted = test(model=sp_activator.model, test_data=test_stock_data, tester=regression_tester, batch_size=64,
                     pre_data_length=pre_data_length, pro_data_length=pro_data_length, process_data=process_data,
                     step_estimator=get_estimated_steps, generator=data_loader)

Total Samples: 1369
14300.0 [2853.1057]
14100.0 [2853.1057]
14000.0 [2853.1057]
14450.0 [2853.1057]
14450.0 [2853.1057]
14100.0 [2853.1057]
14050.0 [2853.1057]
14200.0 [2853.1057]
14400.0 [2853.1057]
14450.0 [2853.1057]
14400.0 [2853.1057]
14550.0 [2853.1057]
14500.0 [2853.1057]
14100.0 [2853.1057]
14200.0 [2853.1057]
14150.0 [2853.1057]
14150.0 [2853.1057]
14200.0 [2853.1057]
14400.0 [2853.1057]
14200.0 [2853.1057]
14050.0 [2853.1057]
14000.0 [2853.1057]
13900.0 [2853.1057]
13700.0 [2853.1057]
13550.0 [2853.1057]
13600.0 [2853.1057]
13650.0 [2853.1057]
13550.0 [2853.1057]
13500.0 [2853.1057]
13650.0 [2853.1057]
13650.0 [2853.1057]
13450.0 [2853.1057]
13250.0 [2853.1057]
13250.0 [2853.1057]
13350.0 [2853.1057]
13400.0 [2853.1057]
14100.0 [2853.1057]
14550.0 [2853.1057]
15050.0 [2853.1057]
14600.0 [2853.1057]
14500.0 [2853.1057]
14850.0 [1295.3517]
14750.0 [1292.149]
14950.0 [1291.6582]
14800.0 [1291.5906]
14850.0 [1291.5814]
14750.0 [1291.5802]
14500.0 [1291.5796]
14500.0 [1291.5591]
1

In [103]:
loader = data_loader(all_stock_data=test_stock_data, batch_size=64, 
                     pre_data_length=20, pro_data_length=1, 
                     process_data=process_data)
for idx, (x_batch, y_batch) in enumerate(loader):
    if idx == 10:
        break
    print(y_batch)

[[50.0]
 [350.0]
 [600.0]
 [200.0]
 [200.0]
 [400.0]
 [800.0]
 [1500.0]
 [100.0]
 [50.0]
 [0.0]
 [500.0]
 [450.0]
 [0.0]
 [800.0]
 [500.0]
 [0.0]
 [100.0]
 [200.0]
 [900.0]
 [350.0]
 [0.0]
 [900.0]
 [100.0]
 [50.0]
 [100.0]
 [900.0]
 [700.0]
 [150.0]
 [0.0]
 [1400.0]
 [500.0]
 [200.0]
 [50.0]
 [1500.0]
 [800.0]
 [50.0]
 [50.0]
 [0.0]
 [1000.0]
 [50.0]
 [600.0]
 [300.0]
 [500.0]
 [150.0]
 [100.0]
 [100.0]
 [600.0]
 [50.0]
 [150.0]
 [0.0]
 [100.0]
 [400.0]
 [0.0]
 [200.0]
 [0.0]
 [100.0]
 [50.0]
 [300.0]
 [1100.0]
 [50.0]
 [0.0]
 [700.0]
 [100.0]]
[[0.0]
 [150.0]
 [200.0]
 [100.0]
 [50.0]
 [250.0]
 [200.0]
 [200.0]
 [200.0]
 [50.0]
 [100.0]
 [100.0]
 [200.0]
 [250.0]
 [200.0]
 [100.0]
 [150.0]
 [400.0]
 [0.0]
 [800.0]
 [50.0]
 [150.0]
 [100.0]
 [500.0]
 [100.0]
 [100.0]
 [100.0]
 [0.0]
 [200.0]
 [100.0]
 [100.0]
 [900.0]
 [150.0]
 [150.0]
 [600.0]
 [800.0]
 [50.0]
 [100.0]
 [100.0]
 [200.0]
 [50.0]
 [50.0]
 [200.0]
 [1000.0]
 [100.0]
 [100.0]
 [100.0]
 [200.0]
 [50.0]
 [100.0]
 [300.0]
 

In [104]:
test_stock_data

[array([[14800.0, 500.0, 15250.0, 15400.0, 14650.0, 89378.0],
        [14900.0, 100.0, 14900.0, 15050.0, 14750.0, 23480.0],
        [14800.0, 100.0, 15100.0, 15100.0, 14700.0, 28288.0],
        ...,
        [12650.0, 400.0, 13050.0, 13100.0, 12550.0, 45778.0],
        [12650.0, 0.0, 12550.0, 12800.0, 12400.0, 23579.0],
        [12900.0, 250.0, 12650.0, 12900.0, 12550.0, 19363.0]], dtype=object),
 array([[13600.0, 350.0, 13800.0, 13950.0, 13550.0, 38914.0],
        [13900.0, 300.0, 13550.0, 13950.0, 13450.0, 44309.0],
        [14050.0, 150.0, 13900.0, 14250.0, 13900.0, 59080.0],
        ...,
        [11750.0, 200.0, 11900.0, 12050.0, 11650.0, 55984.0],
        [12000.0, 250.0, 11900.0, 12150.0, 11750.0, 18309.0],
        [12150.0, 150.0, 12000.0, 12250.0, 12000.0, 14377.0]], dtype=object),
 array([[43700.0, 8400.0, 52100.0, 67000.0, 41100.0, 506129.0],
        [48000.0, 4300.0, 44300.0, 52700.0, 44150.0, 608556.0],
        [54600.0, 6600.0, 49500.0, 56200.0, 47750.0, 258515.0],
        